# Todo List
    - konstruktor
    - Methoden (update, over)
    - Datenstruktur Spielfeld (2D-array)
    - in over() Struktur winningConditions
    - class result
    - class position (pair)
    - tests
    
- __str__ (print weg)
- Position Klasse weg (Indexing direkt in Klasse board(0,1))
- nicht auf instance prüfen, stattdessen auf None
- DRY (schleifen)
- Spalte-Reihe 0,1,2 -> Docstring
    
- später GUI

In [180]:
class Position:
    def __init__(self, pair):
        self.column, self.row = pair
    
    # Convert row to start at index 0 to be usable for numpy array
    def rowConverted(self):
        return self.row-1
    
    # Convert column from letter a,b,c to index 0,1,2 to be usable for numpy array
    # Throws ValueError exception if column is not a,b,c
    def columnAsInt(self):
        if self.column == 'a':
            return 0
        elif self.column == 'b':
            return 1
        elif self.column == 'c':
            return 2
        else:
            raise ValueError("Input column: '" + str(self.column) + "' not supported.")
            
    # Checks if positon is valid
    # Valid: row between 0-2, column between 0-2
    # Returns bool
    def isValid(self):
        if (0 <= self.rowConverted() <= 2) and (0 <= self.columnAsInt() <= 2):
            return True
        return False

In [181]:
# resultType = int -> (-1 || 0 || 1 || 2 || 3)
# -1 = kein result
# 0 = falscher Zug
# 1 = Spieler 1 gewinnt
# 2 = Spieler 2 gewinnt
# 3 = Unentschieden
class Result:
    def __init__(self, resultType): 
        self.resultType = resultType

In [182]:
import numpy as np

# Contains attribute 'board' (3x3 numpy array of type 'Player')
class Board:
    def __init__(self):       
        self.board = np.empty(shape=(3,3), dtype=Player)
    
    # Allow Indexing from outside of the class
    def __getitem__(self, index):
        return self.board[index]
    
    # Update internal numpy array if position is valid
    # Parameter: - Position position: Desired position on the board
    #            - Player player: Currently active player
    def update(self, position, player):
        if not position.isValid():
            return False
        
        # Check if desired position is empty
        if isinstance(self.board[position.columnAsInt()][position.rowConverted()], Player):
            return False
        
        self.markPlayer(position, player)
        return True
    
    # Fill in the currently active player in the numpy array at the desired position
    # and print the updated board
    def markPlayer(self, position, player):
        self.board[position.columnAsInt()][position.rowConverted()] = player
        print(self.board)
    
    # Check if a winning-condition has occured or if no more moves can be made
    # Returns Result (Currently returns the winning player, due to missing Player class)
    def over(self):
        #column winningconditions
        if (self.board[0][0] == self.board[0][1] and self.board[0][0] == self.board[0][2]) and self.board[0][0] != None:
            return self.board[0][0]
        if (self.board[1][0] == self.board[1][1] and self.board[1][0] == self.board[1][2]) and self.board[1][0] != None:
            return self.board[1][0]
        if (self.board[2][0] == self.board[2][1] and self.board[2][0] == self.board[2][2]) and self.board[2][0] != None:
            return self.board[2][0]
        
        #row winningconditions
        if (self.board[0][0] == self.board[1][0] and self.board[0][0] == self.board[2][0]) and self.board[0][0] != None:
            return self.board[0][0]
        if (self.board[0][1] == self.board[1][1] and self.board[0][1] == self.board[2][1]) and self.board[0][1] != None:
            return self.board[0][1]
        if (self.board[0][2] == self.board[1][2] and self.board[0][2] == self.board[2][2]) and self.board[0][2] != None:
            return self.board[0][2]
        
        #diagonal winningcondition
        if (self.board[0][0] == self.board[1][1] and self.board[0][0] == self.board[2][2]) and self.board[0][0] != None:
            return self.board[0][0]
        if (self.board[0][2] == self.board[1][1] and self.board[0][2] == self.board[2][0]) and self.board[0][2] != None:
            return self.board[0][2]
        
        #no one wins
        if self.noMoreMoves():
            return Result(3)
        
        return Result(-1)
    
    # Check if no more moves can be made
    # Returns bool
    def noMoreMoves(self):
        for col, row in np.ndindex(self.board.shape):
            if not isinstance(self.board[col][row], Player):
                return False
        return True
    
    # Clears the internal numpy array
    def clear(self):
        self.board = np.empty(shape=(3,3), dtype=Player)
                

In [183]:
import ipytest
import pytest
ipytest.autoconfig()

In [184]:
%%ipytest

board = Board()

# Temporary Player class until final class is implemented
class Player:
    def __init__(self, _playerNumber):
        self.playerNumber = _playerNumber
    
def test_positon_rowConverted():
    pos = Position(('a',1))
    assert pos.rowConverted() == 0
    
def test_position_columnAsInt():
    pos = Position(('a',1))
    assert pos.columnAsInt() == 0
    pos = Position(('b',1))
    assert pos.columnAsInt() == 1
    pos = Position(('c',1))
    assert pos.columnAsInt() == 2
    
    pos = Position(('d',1))
    with pytest.raises(ValueError):
        pos.isValid()
    
def test_position_isValid():
    pos = Position(('a',1))
    assert pos.isValid() == True
    pos = Position(('b',1))
    assert pos.isValid() == True    
    pos = Position(('c',1))
    assert pos.isValid() == True
    
    pos = Position(('a',2))
    assert pos.isValid() == True
    pos = Position(('b',2))
    assert pos.isValid() == True    
    pos = Position(('c',2))
    assert pos.isValid() == True
    
    pos = Position(('a',3))
    assert pos.isValid() == True
    pos = Position(('b',3))
    assert pos.isValid() == True    
    pos = Position(('c',3))
    assert pos.isValid() == True
        
    pos = Position(('a',0))
    assert pos.isValid() == False
    pos = Position(('a',4))
    assert pos.isValid() == False

    
def test_board_update():
    player1 = Player(1)
    player2 = Player(2)
    
    pos = Position(('a',1))
    assert board.update(pos, player1) == True
    pos = Position(('b',1))
    assert board.update(pos, player2) == True
    pos = Position(('c',1))
    assert board.update(pos, player1) == True
    
    pos = Position(('a',2))
    assert board.update(pos, player2) == True
    pos = Position(('b',2))
    assert board.update(pos, player1) == True
    pos = Position(('c',2))
    assert board.update(pos, player2) == True
    
    pos = Position(('a',3))
    assert board.update(pos, player1) == True
    pos = Position(('b',3))
    assert board.update(pos, player2) == True
    pos = Position(('c',3))
    assert board.update(pos, player1) == True
    
    pos = Position(('a',1))
    assert board.update(pos, player1) == False
    pos = Position(('a',1))
    assert board.update(pos, player2) == False
    
def test_board_markPlayer():
    player = Player(2)
    pos = Position(('a',1))
    board.markPlayer(pos, player)
    assert board[pos.columnAsInt()][pos.rowConverted()] == player
    
def test_board_over():
    board.clear()
    player = Player(1)
    
    result = board.over()
    assert result.resultType == -1
    
    pos = Position(('a',1))
    board.update(pos, player)
    pos = Position(('a',2))
    board.update(pos, player)
    pos = Position(('a',3))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('b',1))
    board.update(pos, player)
    pos = Position(('b',2))
    board.update(pos, player)
    pos = Position(('b',3))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('c',1))
    board.update(pos, player)
    pos = Position(('c',2))
    board.update(pos, player)
    pos = Position(('c',3))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('a',1))
    board.update(pos, player)
    pos = Position(('b',1))
    board.update(pos, player)
    pos = Position(('c',1))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('a',2))
    board.update(pos, player)
    pos = Position(('b',2))
    board.update(pos, player)
    pos = Position(('c',2))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('a',3))
    board.update(pos, player)
    pos = Position(('b',3))
    board.update(pos, player)
    pos = Position(('c',3))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('a',1))
    board.update(pos, player)
    pos = Position(('b',2))
    board.update(pos, player)
    pos = Position(('c',3))
    board.update(pos, player)  
    assert board.over() == player
    
    board.clear()
    pos = Position(('a',3))
    board.update(pos, player)
    pos = Position(('b',2))
    board.update(pos, player)
    pos = Position(('c',1))
    board.update(pos, player)  
    assert board.over() == player
    
def test_board_noMoreMoves():
    player1 = Player(1)
    player2 = Player(2)
    
    assert board.noMoreMoves() == False
    
    pos = Position(('a',1))
    board.update(pos, player1)
    pos = Position(('b',1))
    board.update(pos, player2)
    pos = Position(('c',1))
    board.update(pos, player1)
    
    assert board.noMoreMoves() == False
    
    pos = Position(('a',2))
    board.update(pos, player2)
    pos = Position(('b',2))
    board.update(pos, player1)
    pos = Position(('c',2))
    board.update(pos, player2)
    
    assert board.noMoreMoves() == False
    
    pos = Position(('a',3))
    board.update(pos, player1)
    pos = Position(('b',3))
    board.update(pos, player2)
    pos = Position(('c',3))
    board.update(pos, player1)
    
    assert board.noMoreMoves() == True
    
    
    

.......                                                                                      [100%]
7 passed in 0.02s
